In [7]:
import os

def analyze_coppeliasim_models(base_path):
    """
    Analyze the directory structure and file names for CoppeliaSim models.
    :param base_path: The root directory of the CoppeliaSim models.
    """
    if not os.path.exists(base_path):
        print(f"The path {base_path} does not exist. Please check the installation.")
        return

    print(f"Analyzing model directory: {base_path}\n")

    # Walk through the directory
    for root, dirs, files in os.walk(base_path):
        # Print the current directory
        print(f"Directory: {root}")
        if files:
            print("  Models:")
            for file in files:
                if file.endswith(".ttm"):  # CoppeliaSim model files
                    model_path = os.path.join(root, file)
                    print(f"    - {file} (Path: {model_path})")
        if dirs:
            print("  Subdirectories:")
            for dir in dirs:
                print(f"    - {dir}")
        print("\n")

if __name__ == "__main__":
    # Path to the CoppeliaSim models directory
    base_path = r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models"
    analyze_coppeliasim_models(base_path)


Analyzing model directory: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models

Directory: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models
  Subdirectories:
    - components
    - equipment
    - examples
    - furniture
    - household
    - infrastructure
    - nature
    - office items
    - other
    - people
    - robots
    - tools
    - vehicles


Directory: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\components
  Subdirectories:
    - grippers
    - locomotion and propulsion
    - modifiers
    - sensors


Directory: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\components\grippers
  Models:
    - ABB YuMi gripper.ttm (Path: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\components\grippers\ABB YuMi gripper.ttm)
    - Barrett Hand (simplified).ttm (Path: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\components\grippers\Barrett Hand (simplified).ttm)
    - Barrett Hand.ttm (Path: C:\Program Files\CoppeliaRobotics\CoppeliaSimEd

Environment 1:

In [13]:
import sim
import os
import sys

COPPELIASIM_MODEL_DIR = r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models"

def connect_to_coppeliasim():
    print("Connecting to CoppeliaSim...")
    sim.simxFinish(-1)  # Close any existing connections
    client_id = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
    if client_id == -1:
        print("Failed to connect to CoppeliaSim. Make sure it is running.")
        sys.exit()
    print("Connected successfully.")
    return client_id

def clear_scene(client_id):
    print("Clearing the scene...")

    # Use the default empty scene as a clean start
    empty_scene_path = os.path.join(r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu", "scenes", "default.ttt")
    res = sim.simxLoadScene(client_id, empty_scene_path, 0, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print("Scene cleared successfully.")
    else:
        print("Failed to clear the scene.")

def add_model(client_id, model_path, position=None):
    print(f"Adding model: {model_path}")
    if not os.path.exists(model_path):
        print(f"Model not found: {model_path}")
        sys.exit()
    res, model_handle = sim.simxLoadModel(client_id, model_path, 0, sim.simx_opmode_blocking)
    if res != sim.simx_return_ok:
        print(f"Failed to load model: {model_path}")
        sys.exit()
    print(f"Model added with handle: {model_handle}")

    if position:
        # Set position if provided
        sim.simxSetObjectPosition(client_id, model_handle, -1, position, sim.simx_opmode_blocking)
        print(f"Model moved to position: {position}")
    
    return model_handle

def configure_environment(client_id):
    print("Configuring the environment...")

    # Add Pioneer P3DX robot
    pioneer_path = os.path.join(COPPELIASIM_MODEL_DIR, "robots", "mobile", "pioneer p3dx.ttm")
    add_model(client_id, pioneer_path, position=[0, 0, 0.1])  # Center position

    # Add floor
    floor_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "floors", "5mX5m concrete floor.ttm")
    add_model(client_id, floor_path, position=[0, 0, 0])  # Center floor below the robot

    # Add obstacles (e.g., tables)
    table_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "tables", "conference table.ttm")
    add_model(client_id, table_path, position=[1, 1, 0.75])  # Table near the robot

    # Add shelves
    shelf_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "shelves-cupboards-racks", "rack.ttm")
    add_model(client_id, shelf_path, position=[-1, -1, 0.75])  # Shelf on the other side

    print("Environment configured successfully.")

def save_scene(client_id):
    print("Sending save scene request to CoppeliaSim...")
    # Construct a descriptive save path
    current_dir = os.getcwd()
    save_scene_name = "pioneer_p3dx_with_floor_and_obstacles.ttt"
    save_scene_path = os.path.join(current_dir, save_scene_name)

    # Send the save path to CoppeliaSim via a string signal
    res = sim.simxSetStringSignal(client_id, "saveScenePath", save_scene_path, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print(f"Save request sent for path: {save_scene_path}")
    else:
        print("Failed to send save request.")

def main():
    client_id = connect_to_coppeliasim()
    clear_scene(client_id)
    configure_environment(client_id)
    save_scene(client_id)
    sim.simxFinish(client_id)
    print("Environment setup and saved successfully.")

if __name__ == "__main__":
    main()


Connecting to CoppeliaSim...
Connected successfully.
Clearing the scene...
Failed to clear the scene.
Configuring the environment...
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\robots\mobile\pioneer p3dx.ttm
Model added with handle: 64
Model moved to position: [0, 0, 0.1]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 106
Model moved to position: [0, 0, 0]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\tables\conference table.ttm
Model added with handle: 108
Model moved to position: [1, 1, 0.75]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\shelves-cupboards-racks\rack.ttm
Model added with handle: 110
Model moved to position: [-1, -1, 0.75]
Environment configured successfully.
Sending save scene request to CoppeliaSim...
Save request sent for path: C:\Users\Girija\Documents\GitHub\Msc_AI_Module3_Mobil

Environment 2

In [16]:
import sim
import os
import sys

COPPELIASIM_MODEL_DIR = r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models"

def connect_to_coppeliasim():
    print("Connecting to CoppeliaSim...")
    sim.simxFinish(-1)  # Close any existing connections
    client_id = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
    if client_id == -1:
        print("Failed to connect to CoppeliaSim. Make sure it is running.")
        sys.exit()
    print("Connected successfully.")
    return client_id

def clear_scene(client_id):
    print("Clearing the scene...")

    # Use the default empty scene for a clean start
    empty_scene_path = os.path.join(r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu", "scenes", "default.ttt")
    res = sim.simxLoadScene(client_id, empty_scene_path, 0, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print("Scene cleared successfully.")
    else:
        print("Failed to clear the scene.")

def add_model(client_id, model_path, position):
    """Add a model from the given path to the scene at the specified position."""
    print(f"Adding model: {model_path}")
    if not os.path.exists(model_path):
        print(f"Model not found: {model_path}")
        sys.exit()

    res, model_handle = sim.simxLoadModel(client_id, model_path, 0, sim.simx_opmode_blocking)
    if res != sim.simx_return_ok:
        print(f"Failed to load model: {model_path}")
        sys.exit()
    
    sim.simxSetObjectPosition(client_id, model_handle, -1, position, sim.simx_opmode_blocking)
    print(f"Model added with handle: {model_handle}, moved to position: {position}")
    return model_handle

def configure_environment(client_id):
    print("Configuring the environment...")

    # 1. Add Pioneer P3DX robot in the center
    pioneer_path = os.path.join(COPPELIASIM_MODEL_DIR, "robots", "mobile", "pioneer p3dx.ttm")
    add_model(client_id, pioneer_path, position=[0, 0, 0.1])  # Center position

    # 2. Add a 5m x 5m floor
    floor_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "floors", "5mX5m concrete floor.ttm")
    add_model(client_id, floor_path, position=[0, 0, 0])

    # 3. Add tables as obstacles
    table_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "tables", "conference table.ttm")
    add_model(client_id, table_path, position=[1, 1, 0.75])  # Table 1
    add_model(client_id, table_path, position=[-1.5, 1.5, 0.75])  # Table 2

    # 4. Add shelves for complexity
    shelf_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "shelves-cupboards-racks", "rack.ttm")
    add_model(client_id, shelf_path, position=[-2, -2, 0.75])  # Shelf 1
    add_model(client_id, shelf_path, position=[1.5, -1.5, 0.75])  # Shelf 2

    # 5. Add walls to create a boundary
    wall_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "walls", "external wall.ttm")
    add_model(client_id, wall_path, position=[2.5, 0, 1])  # Right wall
    add_model(client_id, wall_path, position=[-2.5, 0, 1])  # Left wall
    add_model(client_id, wall_path, position=[0, 2.5, 1])  # Top wall
    add_model(client_id, wall_path, position=[0, -2.5, 1])  # Bottom wall

    # 6. Add a conveyor belt to simulate warehouse operations
    conveyor_path = os.path.join(COPPELIASIM_MODEL_DIR, "equipment", "conveyors", "generic conveyor (belt).ttm")
    add_model(client_id, conveyor_path, position=[0, -1, 0.1])

    # 7. Add a chair as a static obstacle
    chair_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "chairs", "swivel chair.ttm")
    add_model(client_id, chair_path, position=[0.5, 0.5, 0.4])

    # 8. Add a box for more navigation complexity
    box_path = os.path.join(COPPELIASIM_MODEL_DIR, "household", "largeBasket.ttm")
    add_model(client_id, box_path, position=[-1, 0, 0.25])

    # 9. Add a path marker to guide the robot
    marker_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "other", "resizable concret block.ttm")
    add_model(client_id, marker_path, position=[0, 1, 0.05])

    print("Environment configured successfully.")

def save_scene(client_id):
    print("Sending save scene request to CoppeliaSim...")
    # Construct a descriptive save path
    current_dir = os.getcwd()
    save_scene_name = "complex_environment_pioneer_p3dx.ttt"
    save_scene_path = os.path.join(current_dir, save_scene_name)

    # Send the save path to CoppeliaSim via a string signal
    res = sim.simxSetStringSignal(client_id, "saveScenePath", save_scene_path, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print(f"Save request sent for path: {save_scene_path}")
    else:
        print("Failed to send save request.")

def main():
    client_id = connect_to_coppeliasim()
    clear_scene(client_id)
    configure_environment(client_id)
    save_scene(client_id)
    sim.simxFinish(client_id)
    print("Complex environment setup and saved successfully.")

if __name__ == "__main__":
    main()


Connecting to CoppeliaSim...
Connected successfully.
Clearing the scene...
Failed to clear the scene.
Configuring the environment...
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\robots\mobile\pioneer p3dx.ttm
Model added with handle: 278, moved to position: [0, 0, 0.1]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 320, moved to position: [0, 0, 0]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\tables\conference table.ttm
Model added with handle: 322, moved to position: [1, 1, 0.75]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\tables\conference table.ttm
Model added with handle: 324, moved to position: [-1.5, 1.5, 0.75]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\shelves-cupboards-racks\rack.ttm
Model added with handle: 326, moved to position: [-2, -2, 0.75]
Adding 

Environment 3

In [29]:
import sim
import os
import sys

# Path to the CoppeliaSim model directory
COPPELIASIM_MODEL_DIR = r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models"

def connect_to_coppeliasim():
    """Connect to CoppeliaSim using the remote API."""
    print("Connecting to CoppeliaSim...")
    sim.simxFinish(-1)  # Close any existing connections
    client_id = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
    if client_id == -1:
        print("Failed to connect to CoppeliaSim. Make sure it is running.")
        sys.exit()
    print("Connected successfully.")
    return client_id

def clear_scene(client_id):
    """Clear the current scene to start with an empty environment."""
    print("Clearing the scene...")

    # Use the default empty scene for a clean start
    empty_scene_path = os.path.join(r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu", "scenes", "default.ttt")
    res = sim.simxLoadScene(client_id, empty_scene_path, 0, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print("Scene cleared successfully.")
    else:
        print("Failed to clear the scene.")

def add_model(client_id, model_path, position):
    """Add a model from the given path to the scene at the specified position."""
    print(f"Adding model: {model_path}")
    if not os.path.exists(model_path):
        print(f"Model not found: {model_path}")
        sys.exit()

    res, model_handle = sim.simxLoadModel(client_id, model_path, 0, sim.simx_opmode_blocking)
    if res != sim.simx_return_ok:
        print(f"Failed to load model: {model_path}")
        sys.exit()

    sim.simxSetObjectPosition(client_id, model_handle, -1, position, sim.simx_opmode_blocking)
    print(f"Model added with handle: {model_handle}, moved to position: {position}")
    return model_handle

def configure_environment(client_id):
    """Configure a new environment in the cleared scene."""
    print("Configuring the environment...")

    # 1. Add Pioneer P3DX robot in the center
    pioneer_path = os.path.join(COPPELIASIM_MODEL_DIR, "robots", "mobile", "pioneer p3dx.ttm")
    add_model(client_id, pioneer_path, position=[0, 0, 0.1])  # Center position

    # 2. Add a 5m x 5m floor
    floor_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "floors", "5mX5m concrete floor.ttm")
    add_model(client_id, floor_path, position=[0, 0, 0])

    # 3. Add tables as obstacles
    table_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "tables", "conference table.ttm")
    add_model(client_id, table_path, position=[1, 1, 0.75])  # Table 1
    add_model(client_id, table_path, position=[-1.5, 1.5, 0.75])  # Table 2

    # 4. Add shelves for complexity
    shelf_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "shelves-cupboards-racks", "rack.ttm")
    add_model(client_id, shelf_path, position=[-2, -2, 0.75])  # Shelf 1
    add_model(client_id, shelf_path, position=[1.5, -1.5, 0.75])  # Shelf 2

    # 5. Add walls to create a boundary
    wall_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "walls", "external wall.ttm")
    add_model(client_id, wall_path, position=[2.5, 0, 1])  # Right wall
    add_model(client_id, wall_path, position=[-2.5, 0, 1])  # Left wall
    add_model(client_id, wall_path, position=[0, 2.5, 1])  # Top wall
    add_model(client_id, wall_path, position=[0, -2.5, 1])  # Bottom wall

    # 6. Add a conveyor belt to simulate warehouse operations
    conveyor_path = os.path.join(COPPELIASIM_MODEL_DIR, "equipment", "conveyors", "generic conveyor (belt).ttm")
    add_model(client_id, conveyor_path, position=[0, -1, 0.1])

    # 7. Add a chair as a static obstacle
    chair_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "chairs", "swivel chair.ttm")
    add_model(client_id, chair_path, position=[0.5, 0.5, 0.4])

    # 8. Add a box for more navigation complexity
    box_path = os.path.join(COPPELIASIM_MODEL_DIR, "household", "largeBasket.ttm")
    add_model(client_id, box_path, position=[-1, 0, 0.25])

    # 9. Add a path marker to guide the robot
    marker_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "other", "resizable concret block.ttm")
    add_model(client_id, marker_path, position=[0, 1, 0.05])

    print("Environment configured successfully.")

def save_scene(client_id):
    """Save the current scene in CoppeliaSim."""
    print("Sending save scene request to CoppeliaSim...")
    # Construct a descriptive save path
    current_dir = os.getcwd()
    save_scene_name = "complex_environment_pioneer_p3dx.ttt"
    save_scene_path = os.path.join(current_dir, save_scene_name)

    # Send the save path to CoppeliaSim via a string signal
    res = sim.simxSetStringSignal(client_id, "saveScenePath", save_scene_path, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print(f"Save request sent for path: {save_scene_path}")
    else:
        print("Failed to send save request.")

def main():
    """Main function to set up the simulation environment."""
    client_id = connect_to_coppeliasim()
    clear_scene(client_id)
    configure_environment(client_id)
    save_scene(client_id)
    sim.simxFinish(client_id)
    print("Complex environment setup and saved successfully.")

if __name__ == "__main__":
    main()


Connecting to CoppeliaSim...
Connected successfully.
Clearing the scene...
Failed to clear the scene.
Configuring the environment...
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\robots\mobile\pioneer p3dx.ttm
Model added with handle: 57, moved to position: [0, 0, 0.1]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 99, moved to position: [0, 0, 0]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\tables\conference table.ttm
Model added with handle: 101, moved to position: [1, 1, 0.75]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\tables\conference table.ttm
Model added with handle: 103, moved to position: [-1.5, 1.5, 0.75]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\furniture\shelves-cupboards-racks\rack.ttm
Model added with handle: 105, moved to position: [-2, -2, 0.75]
Adding mo

In [32]:
import sim
import os
import sys

# Path to the CoppeliaSim model directory
COPPELIASIM_MODEL_DIR = r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models"

def connect_to_coppeliasim():
    """Connect to CoppeliaSim using the remote API."""
    print("Connecting to CoppeliaSim...")
    sim.simxFinish(-1)  # Close any existing connections
    client_id = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
    if client_id == -1:
        print("Failed to connect to CoppeliaSim. Make sure it is running.")
        sys.exit()
    print("Connected successfully.")
    return client_id

def clear_scene(client_id):
    """Clear the current scene to start with an empty environment."""
    print("Clearing the scene...")
    # Use the default empty scene for a clean start
    empty_scene_path = os.path.join(r"C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu", "scenes", "default.ttt")
    res = sim.simxLoadScene(client_id, empty_scene_path, 0, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print("Scene cleared successfully.")
    else:
        print("Failed to clear the scene.")

def add_model(client_id, model_path, position):
    """Add a model from the given path to the scene at the specified position."""
    print(f"Adding model: {model_path}")
    if not os.path.exists(model_path):
        print(f"Model not found: {model_path}")
        sys.exit()

    res, model_handle = sim.simxLoadModel(client_id, model_path, 0, sim.simx_opmode_blocking)
    if res != sim.simx_return_ok:
        print(f"Failed to load model: {model_path}")
        sys.exit()

    sim.simxSetObjectPosition(client_id, model_handle, -1, position, sim.simx_opmode_blocking)
    print(f"Model added with handle: {model_handle}, moved to position: {position}")
    return model_handle

def configure_complete_house(client_id):
    """Configure a complete house-like environment in the cleared scene."""
    print("Configuring the complete house environment...")

    # 1. Add Pioneer P3DX robot in the hallway (starting position)
    pioneer_path = os.path.join(COPPELIASIM_MODEL_DIR, "robots", "mobile", "pioneer p3dx.ttm")
    add_model(client_id, pioneer_path, position=[0, 0, 0.1])

    # 2. Add floors for each room
    floor_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "floors", "5mX5m concrete floor.ttm")
    add_model(client_id, floor_path, position=[0, 0, 0])  # Living room
    add_model(client_id, floor_path, position=[5, 0, 0])  # Kitchen
    add_model(client_id, floor_path, position=[0, 5, 0])  # Bedroom
    add_model(client_id, floor_path, position=[5, 5, 0])  # Dining room

    # 3. Add walls for the house
    wall_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "walls", "external wall.ttm")
    # Living room walls
    add_model(client_id, wall_path, position=[-2.5, 0, 1])  # Left wall
    add_model(client_id, wall_path, position=[2.5, 0, 1])   # Right wall
    add_model(client_id, wall_path, position=[0, 2.5, 1])   # Top wall
    # Kitchen walls
    add_model(client_id, wall_path, position=[7.5, 0, 1])   # Right wall
    add_model(client_id, wall_path, position=[5, 2.5, 1])   # Top wall
    # Bedroom walls
    add_model(client_id, wall_path, position=[-2.5, 7.5, 1]) # Left wall
    add_model(client_id, wall_path, position=[0, 7.5, 1])   # Top wall
    # Dining room walls
    add_model(client_id, wall_path, position=[7.5, 7.5, 1]) # Right wall
    add_model(client_id, wall_path, position=[5, 7.5, 1])   # Top wall

    # 4. Add doors between rooms
    door_path = os.path.join(COPPELIASIM_MODEL_DIR, "infrastructure", "doors", "manual door.ttm")
    add_model(client_id, door_path, position=[2.5, 2, 1])   # Door between living room and kitchen
    add_model(client_id, door_path, position=[-2.5, 5, 1])  # Door between living room and bedroom
    add_model(client_id, door_path, position=[5, 5, 1])     # Door between kitchen and dining room

    # 5. Add furniture to different rooms
    # Living room furniture
    sofa_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "chairs", "sofa.ttm")
    add_model(client_id, sofa_path, position=[-1, -1, 0.4])

    coffee_table_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "tables", "conference table.ttm")
    add_model(client_id, coffee_table_path, position=[0, -2, 0.35])

    # Kitchen furniture
    kitchen_cabinet_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "shelves-cupboards-racks", "deep cupboard.ttm")
    add_model(client_id, kitchen_cabinet_path, position=[6, -1, 0.8])

    fridge_path = os.path.join(COPPELIASIM_MODEL_DIR, "household", "largeBasket.ttm")
    add_model(client_id, fridge_path, position=[6, 1, 0.9])

    # Bedroom furniture
    bed_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "tables", "high table.ttm")
    add_model(client_id, bed_path, position=[-1, 6, 0.4])

    wardrobe_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "shelves-cupboards-racks", "rack.ttm")
    add_model(client_id, wardrobe_path, position=[-3, 6, 0.9])

    # Dining room furniture
    dining_table_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "tables", "diningTable.ttm")
    add_model(client_id, dining_table_path, position=[5, 6, 0.75])

    dining_chair_path = os.path.join(COPPELIASIM_MODEL_DIR, "furniture", "chairs", "dining chair.ttm")
    add_model(client_id, dining_chair_path, position=[5, 7, 0.4])
    add_model(client_id, dining_chair_path, position=[4, 6, 0.4])
    add_model(client_id, dining_chair_path, position=[6, 6, 0.4])
    add_model(client_id, dining_chair_path, position=[5, 5, 0.4])

    # 6. Add obstacles for additional complexity
    small_bucket_path = os.path.join(COPPELIASIM_MODEL_DIR, "household", "largeBasket.ttm")
    add_model(client_id, small_bucket_path, position=[1, 1, 0.25])  # Living room
    add_model(client_id, small_bucket_path, position=[6, 4, 0.25])  # Kitchen

    print("House environment configured successfully.")

def save_scene(client_id):
    """Save the current scene in CoppeliaSim."""
    print("Sending save scene request to CoppeliaSim...")
    # Construct a descriptive save path
    current_dir = os.getcwd()
    save_scene_name = "complete_house_environment_pioneer_p3dx.ttt"
    save_scene_path = os.path.join(current_dir, save_scene_name)

    # Send the save path to CoppeliaSim via a string signal
    res = sim.simxSetStringSignal(client_id, "saveScenePath", save_scene_path, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print(f"Save request sent for path: {save_scene_path}")
    else:
        print("Failed to send save request.")

def main():
    """Main function to set up the complete house-like simulation environment."""
    client_id = connect_to_coppeliasim()
    clear_scene(client_id)
    configure_complete_house(client_id)
    save_scene(client_id)
    sim.simxFinish(client_id)
    print("Complete house environment setup and saved successfully.")

if __name__ == "__main__":
    main()


Connecting to CoppeliaSim...
Connected successfully.
Clearing the scene...
Failed to clear the scene.
Configuring the complete house environment...
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\robots\mobile\pioneer p3dx.ttm
Model added with handle: 251, moved to position: [0, 0, 0.1]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 293, moved to position: [0, 0, 0]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 295, moved to position: [5, 0, 0]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 297, moved to position: [0, 5, 0]
Adding model: C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\models\infrastructure\floors\5mX5m concrete floor.ttm
Model added with handle: 299, moved to positi